#### Login using the gcloud CLI

In [ ]:
!gcloud auth login

#### Set the project that contains your bucket

In [ ]:
# !gcloud config set project project-id-1
!gcloud config set project project-id-2

#### CREATE A SERVICE ACCOUNT | just in case you dont have one

In [ ]:
# !gcloud iam service-accounts create readonly-data-viewer --description="Service account for read-only access to Datastore" --display-name="Datastore ReadOnly"

#### Set the service account permissions

In [ ]:
# !gcloud projects add-iam-policy-binding project-id-2 --member=serviceAccount:readonly-data-viewer@project-id-2.iam.gserviceaccount.com --role=roles/storage.objectViewer
# !gcloud projects add-iam-policy-binding project-id-2 --member=serviceAccount:readonly-data-viewer@project-id-2.iam.gserviceaccount.com --role=roles/storage.objectCreator

#### Get the keys

In [ ]:
# !gcloud iam service-accounts keys create ./datastore-readonly3-key.json --iam-account readonly-data-viewer@project-id-2.iam.gserviceaccount.com

#### Once you conclude the work, remove risky permissions

In [ ]:
# !gcloud projects remove-iam-policy-binding steam-project-id-2 --member=serviceAccount:readonly-data-viewer@steam-project-id-2.iam.gserviceaccount.com --role=roles/storage.objectCreator

#### Let's begin the game

In [ ]:
import tempfile
from glob import glob
from google.cloud import storage

#### The functions I created for this task


In [ ]:
def get_files(bucket_name):
    blobs = storage_client.list_blobs(bucket_name)  ### Bucket initialization
    files = [blob.name for blob in blobs]           ### Get list of blobs in bucket
    return files

def download_tmp_file(storage_client, bucket_name, blob_source, namef=''):
    # Download files from bucket
    bucket = storage_client.bucket(bucket_name) # Initialize bucket
    blob = bucket.blob(blob_source)             # Initialize blob
    if namef!="":temp_local_path=f"/tmp/{namef}"
    else:_, temp_local_path = tempfile.mkstemp()     # Create a temporary file
    print(temp_local_path)
    folder = "/".join(temp_local_path.split("/")[:-1])
    os.makedirs(folder, exist_ok=True)
    blob.download_to_filename(temp_local_path)  # Download blob to temp file
    return temp_local_path

def upload_file(bucket_name, local_file, dest_buck_file):
    bucket = storage_client.bucket(bucket_name) ### bucket initialize
    blob = bucket.blob(dest_buck_file)          ### destiny directory in the bucket
    blob.upload_from_filename(local_file)       ### local filename]

#### Set the location of your keys

In [ ]:
import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/datastore-readonly2-key.json" ### keys from account 1
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/datastore-readonly3-key.json"   ### keys from account 2

#### Initialize your bucket

In [ ]:
storage_client = storage.Client()

# bucket_name = 'bucket_name_1'
bucket_name = 'bucket_name_2'
blobs = storage_client.list_blobs(bucket_name)

#### Get the old bucket files into a list, remove those files you don't want to migrate

In [ ]:
L = get_files(bucket_name=bucket_name)
L

#### Download the files into the temp folder in the Google Colab session

In [ ]:
for filee in L:
    download_tmp_file(storage_client=storage_client,
                      bucket_name=bucket_name,
                      blob_source=filee,
                      namef=f'bucket/{filee}'
                      )

#### Finally, upload those files into the new bucket. Remember to set the required permissions to the service account.

In [ ]:
for ff in glob("/tmp/bucket/**/*.*", recursive=True):
    # print(ff)
    dest = ff.replace("/tmp/bucket/", "")
    print(ff, dest)
    upload_file(bucket_name=bucket_name,
                local_file=ff,
                dest_buck_file=dest)

#### And that's it. Now you're a professional Google Cloud Storage migrator!